# Avg Property Value

### Logic:
* Step 1: Use Tax Rate Table to get assessed value.
    * Assessed Value=Property Tax/Tax Rate 
* Step 2: Use Table of Equalized Valuations (TEV)to get value.
    * Property Value = Assessed Value/Equalized Value Ratio 
* Step 3. Group data by:
    * County-Town-Zip to get average property value by zip code.
    * County-Town to get average property valye by town. 

In [2]:
#Import Dependencies
import pandas as pd
import os

In [3]:
#Read Owner Occupied Pickle File
input_path_owner = os.path.join("..","Final Output Data","OwnerOccupiedProperty_Data.plk")
input_path_tenant= os.path.join("..","Final Output Data","TenantOccupied_Data.plk")
input_path_tax_rate= os.path.join("","raw data","Tax_Rate_TEV.csv")

In [4]:
#Get Owner Data
owner_df= pd.read_pickle(input_path_owner)
owner_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
1,2019,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73,07922
3,2019,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95,07657
6,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,213600,249300,462900,10631.31,08854
8,2019,EAST HANOVER TWP,MORRIS,RESIDENTIAL,189100,292200,481300,10949.58,07936
9,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,258500,596700,855200,19134.93,08854


In [21]:
#Check data size
owner_df.size

19262150

In [5]:
#Read Tenant Occupied Pickle file
tenant_df= pd.read_pickle(input_path_tenant)
tenant_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
5,2019,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22,08203
7,2019,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96,08753
11,2019,SOMERS POINT CITY,ATLANTIC,RESIDENTIAL,68500,171100,229300,7330.13,08244
16,2019,VENTNOR CITY,ATLANTIC,RESIDENTIAL,80000,30000,110000,3405.76,08406
22,2019,GREENWICH TWP,WARREN,RESIDENTIAL,31800,85300,117100,3791.70,08323


In [22]:
#Check data size
tenant_df.size

6403260

In [6]:
#Read Tax Rate and Town Equivalance Ratio Value (TEV) CSV File
tax_rate_tev_df = pd.read_csv(input_path_tax_rate)

In [7]:
#Function to parse address line
def partAddress(x):
    y=str(x).split(" ")
    if len(y)>=2:
        return y[0]+y[1]
    else:
        return y[0]

In [8]:
#Create Key Using County Name and Partial Address
tenant_add_part = tenant_df["MUNICIPALITY"].apply(partAddress)
tenant_df["County and Partial Address"] = tenant_df["COUNTY"] + tenant_add_part
tenant_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code,County and Partial Address
5,2019,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22,08203,ATLANTICBRIGANTINECITY
7,2019,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96,08753,OCEANTOMSRIVER
11,2019,SOMERS POINT CITY,ATLANTIC,RESIDENTIAL,68500,171100,229300,7330.13,08244,ATLANTICSOMERSPOINT
16,2019,VENTNOR CITY,ATLANTIC,RESIDENTIAL,80000,30000,110000,3405.76,08406,ATLANTICVENTNORCITY
22,2019,GREENWICH TWP,WARREN,RESIDENTIAL,31800,85300,117100,3791.70,08323,WARRENGREENWICHTWP


In [9]:
#Create Key Using County Name and Partial Address
owner_add_part = owner_df["MUNICIPALITY"].apply(partAddress)
owner_df["County and Partial Address"] = owner_df["COUNTY"] + owner_add_part
owner_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code,County and Partial Address
1,2019,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73,07922,UNIONBERKELEYHEIGHTS
3,2019,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95,07657,BERGENRIDGEFIELDBORO
6,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,213600,249300,462900,10631.31,08854,MIDDLESEXPISCATAWAYTWP
8,2019,EAST HANOVER TWP,MORRIS,RESIDENTIAL,189100,292200,481300,10949.58,07936,MORRISEASTHANOVER
9,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,258500,596700,855200,19134.93,08854,MIDDLESEXPISCATAWAYTWP


In [10]:
#Create Key Using County Name and Partial Address
tev_add_part = tax_rate_tev_df["District"].apply(partAddress)
tax_rate_tev_df["County and Partial Address"] = tax_rate_tev_df["County"] + tev_add_part
tax_rate_tev_df.head()

,County,Cty/District Code,District,TAX RATE,TEV,County and Partial Address
0,ATLANTIC,101,ABSECON CITY,3.292,93.96,ATLANTICABSECONCITY
1,ATLANTIC,102,ATLANTIC CITY,3.533,104.54,ATLANTICATLANTICCITY
2,ATLANTIC,103,BRIGANTINE CITY,1.737,99.20,ATLANTICBRIGANTINECITY
3,ATLANTIC,104,BUENA BORO,3.107,112.40,ATLANTICBUENABORO
4,ATLANTIC,105,BUENA VISTA TWP,2.479,111.40,ATLANTICBUENAVISTA


In [11]:
#Create new Owner Occupied DF with Tax Rate and TEV
owner_tev_df = pd.merge(owner_df,
                 tax_rate_tev_df,
                 on='County and Partial Address', 
                 how='left')
owner_tev_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code,County and Partial Address,County,Cty/District Code,District,TAX RATE,TEV
0,2019,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73,07922,UNIONBERKELEYHEIGHTS,UNION,2001,BERKELEY HEIGHTS TWP,4.090,53.08
1,2019,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95,07657,BERGENRIDGEFIELDBORO,BERGEN,249,RIDGEFIELD BORO,2.417,80.66
2,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,213600,249300,462900,10631.31,08854,MIDDLESEXPISCATAWAYTWP,MIDDLESEX,1217,PISCATAWAY TWP,2.304,93.67
3,2019,EAST HANOVER TWP,MORRIS,RESIDENTIAL,189100,292200,481300,10949.58,07936,MORRISEASTHANOVER,MORRIS,1410,EAST HANOVER TWP,2.275,75.40
4,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,258500,596700,855200,19134.93,08854,MIDDLESEXPISCATAWAYTWP,MIDDLESEX,1217,PISCATAWAY TWP,2.304,93.67


In [15]:
#Calculate Property Market Value by Prior Year Tax Amount by Tax Rate and applying TEV Factor
owner_tev_df["Property Market Value"] = (owner_tev_df["PRIOR YEAR TAX AMOUNT"] / (owner_tev_df["TAX RATE"] / 100)) / (owner_tev_df["TEV"] / 100)
owner_tev_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code,County and Partial Address,County,Cty/District Code,District,TAX RATE,TEV,Property Market Value
0,2019,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73,07922,UNIONBERKELEYHEIGHTS,UNION,2001,BERKELEY HEIGHTS TWP,4.090,53.08,319706.103994
1,2019,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95,07657,BERGENRIDGEFIELDBORO,BERGEN,249,RIDGEFIELD BORO,2.417,80.66,568435.664354
2,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,213600,249300,462900,10631.31,08854,MIDDLESEXPISCATAWAYTWP,MIDDLESEX,1217,PISCATAWAY TWP,2.304,93.67,492610.638856
3,2019,EAST HANOVER TWP,MORRIS,RESIDENTIAL,189100,292200,481300,10949.58,07936,MORRISEASTHANOVER,MORRIS,1410,EAST HANOVER TWP,2.275,75.40,638329.203953
4,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,258500,596700,855200,19134.93,08854,MIDDLESEXPISCATAWAYTWP,MIDDLESEX,1217,PISCATAWAY TWP,2.304,93.67,886632.982367


In [20]:
#Verify data size and should match (19262150)
owner_tev_df.size

31074480

In [16]:
#Create new Owner Occupied DF with Tax Rate and TEV
tenant_tev_df = pd.merge(tenant_df,
                 tax_rate_tev_df,
                 on='County and Partial Address', 
                 how='left')
tenant_tev_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code,County and Partial Address,County,Cty/District Code,District,TAX RATE,TEV
0,2019,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22,08203,ATLANTICBRIGANTINECITY,ATLANTIC,103,BRIGANTINE CITY,1.737,99.20
1,2019,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96,08753,OCEANTOMSRIVER,OCEAN,9999,TOMS RIVER,2.903,81.57
2,2019,SOMERS POINT CITY,ATLANTIC,RESIDENTIAL,68500,171100,229300,7330.13,08244,ATLANTICSOMERSPOINT,ATLANTIC,121,SOMERS POINT CITY,3.067,101.09
3,2019,VENTNOR CITY,ATLANTIC,RESIDENTIAL,80000,30000,110000,3405.76,08406,ATLANTICVENTNORCITY,ATLANTIC,122,VENTNOR CITY,2.634,91.78
4,2019,GREENWICH TWP,WARREN,RESIDENTIAL,31800,85300,117100,3791.70,08323,WARRENGREENWICHTWP,WARREN,2107,GREENWICH TWP,3.238,97.76


In [18]:
#Calculate Property Market Value by Prior Year Tax Amount by Tax Rate and applying TEV Factor
tenant_tev_df["Property Market Value"] = (tenant_tev_df["PRIOR YEAR TAX AMOUNT"] / (tenant_tev_df["TAX RATE"] / 100)) / (tenant_tev_df["TEV"] / 100)
tenant_tev_df.head()

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code,County and Partial Address,County,Cty/District Code,District,TAX RATE,TEV,Property Market Value
0,2019,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22,08203,ATLANTICBRIGANTINECITY,ATLANTIC,103,BRIGANTINE CITY,1.737,99.20,477116.877449
1,2019,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96,08753,OCEANTOMSRIVER,OCEAN,9999,TOMS RIVER,2.903,81.57,206672.606758
2,2019,SOMERS POINT CITY,ATLANTIC,RESIDENTIAL,68500,171100,229300,7330.13,08244,ATLANTICSOMERSPOINT,ATLANTIC,121,SOMERS POINT CITY,3.067,101.09,236422.989415
3,2019,VENTNOR CITY,ATLANTIC,RESIDENTIAL,80000,30000,110000,3405.76,08406,ATLANTICVENTNORCITY,ATLANTIC,122,VENTNOR CITY,2.634,91.78,140880.283362
4,2019,GREENWICH TWP,WARREN,RESIDENTIAL,31800,85300,117100,3791.70,08323,WARRENGREENWICHTWP,WARREN,2107,GREENWICH TWP,3.238,97.76,119783.205571


In [19]:
#Verify data size and shoukd match 6403260
tenant_tev_df.size

10378720